# Introduction

### SFRCP: Data Analysis

This script will be split into different sections for simplicity, with comments explaining each section of code.


# Imports and Setup

In [ ]:
import csv
import numpy as np

prefix = 'saved raw data/'
FUV = 'FUV/'
NUV = 'NUV/'
w1 = 'w1/'
w3 = 'w3/'

In [ ]:
galaxies_list = [
    'NGC 300',
    'M 99',
    'NGC 3938',
    'NGC 3184',
    'M 100', 
    'M 74',
    'NGC 1304',
    'NGC 3265',
    'NGC 855',
    'NGC 1404', 
    'NGC 584',
    'NGC 5866'
]

with open('GALAXY INFO.csv') as csvfile: galaxy_info = list(csv.reader(csvfile))

raw_data_list = np.empty(len(galaxies_list) * 2, dtype=np.dtype('U35'))

for i in range(len(raw_data_list)):   
    if i < 12:
        if galaxies_list[i] == galaxy_info[i + 1][0]:
            gal_type = galaxy_info[i + 1][3]
            mean_gal = galaxies_list[i] + '-raw_mean.csv'
            
            if gal_type == 'Spiral': raw_data_list[i] = 'spiral/' + mean_gal            
            if gal_type == 'Elliptical': raw_data_list[i] = 'elliptical/' + mean_gal
    else:
        i2 =  i - 12
        if galaxies_list[i2] == galaxy_info[i2 + 1][0]:
            gal_type = galaxy_info[i2 + 1][3]
            sum_gal = galaxies_list[i2] + '-raw_sum.csv'

            if gal_type == 'Spiral': raw_data_list[i] = 'spiral/' + sum_gal
            if gal_type == 'Elliptical': raw_data_list[i] = 'elliptical/' + sum_gal

w1_raw_data_list = [w1 + sub for sub in raw_data_list]
w3_raw_data_list = [w3 + sub for sub in raw_data_list]
FUV_raw_data_list = [FUV + sub for sub in raw_data_list]
NUV_raw_data_list = [NUV + sub for sub in raw_data_list]

wise_raw_data_list = w1_raw_data_list + w3_raw_data_list
galex_raw_data_list = FUV_raw_data_list + NUV_raw_data_list

# full_raw_data_list = wise_raw_data_list + galex_raw_data_list

# BUT HOW WILL ANALYSIS WORK THIS TIME? SUPER GALAXIES? COMPARE SLOPES? IDK YET

In [ ]:
# code goes here